# SQL Project
You were hired by Ironhack to perform an Analytics Consulting Project entitled: competitive landscape.

Your mission is to create and populate an appropriate database with many coding schools that are our competition, as well as design an suitable queries that answer business questions of interest (to be defined by you)


**Suggested Steps in the Project:**


*   Read this notebook and understand each function. Comment the code appropriately

*   Populate the list of schools with a wider variety of schools (how are you going to get the school ID?)

* Take a look at the obtained dataframes. What dimensions do you have? what keys do you have? how could the different dataframes be connected?

* Go back to the drawing board and try to create an entity relationship diagram for tables available

* Once you have the schemas you want, you will need to:
  - create the suitable SQL queries to create the tables and populate them
  - run these queries using the appropriate Python connectors
  
* Bonus: How will this datamodel be updated in the future? Please write auxiliary functions that test the database for data quality issues. For example: how could you make sure you only include the most recent comments when you re-run the script?


# Suggested Deliverables

* 5-6 minute presentation of data model created, decision process and business analysis proposed

* exported .sql file with the final schema

* Supporting python files used to generate all logic

* High level documentation explaining tables designed and focusing on update methods

Crucial hint: check out the following tutorial:
https://www.dataquest.io/blog/sql-insert-tutorial/


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pandas import json_normalize
import re
import numpy as np
import time
import multiprocessing
import concurrent

In [2]:
school_list = ['Springboard',
             'Dataquest',
             'Syntax Technologies',
             'ironhack',
             'tripleten',
             'Colaberry',
             'Maven Analytics',
             'Udacity',
             'BrainStation',
             'CCS Learning Academy',
             'Thinkful',
             'General Assembly']

In [3]:
school_id_dict = {}

In [4]:
for school_name in school_list:
    url = f"https://www.switchup.org/bootcamps/{school_name.lower().replace(' ', '-')}"

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        page_data_tag = soup.find('page-data')

        if page_data_tag:
            school_id = page_data_tag.get('school-id')
            if school_id:
                school_id_dict[school_name.lower().replace(' ','-')] = int(school_id)
                print(f"school ID for : {school_name}/{school_id}")
            else:
                print(f"School ID not found: {school_name}")
        else:
            print(f"Page data not found: {school_name}")

    else:
        print(f"Error: {response.status_code}")

print("School ID dictionary:", school_id_dict)

school ID for : Springboard/11035
school ID for : Dataquest/10683
school ID for : Syntax Technologies/11797
school ID for : ironhack/10828
school ID for : tripleten/11225
school ID for : Colaberry/11718
school ID for : Maven Analytics/11740
school ID for : Udacity/11118
school ID for : BrainStation/10571
school ID for : CCS Learning Academy/11736
school ID for : Thinkful/11098
school ID for : General Assembly/10761
School ID dictionary: {'springboard': 11035, 'dataquest': 10683, 'syntax-technologies': 11797, 'ironhack': 10828, 'tripleten': 11225, 'colaberry': 11718, 'maven-analytics': 11740, 'udacity': 11118, 'brainstation': 10571, 'ccs-learning-academy': 11736, 'thinkful': 11098, 'general-assembly': 10761}


In [5]:
school_id_dict

{'springboard': 11035,
 'dataquest': 10683,
 'syntax-technologies': 11797,
 'ironhack': 10828,
 'tripleten': 11225,
 'colaberry': 11718,
 'maven-analytics': 11740,
 'udacity': 11118,
 'brainstation': 10571,
 'ccs-learning-academy': 11736,
 'thinkful': 11098,
 'general-assembly': 10761}

In [6]:
def get_comments_school(school):
    TAG_RE = re.compile(r'<[^>]+>')
    # defines url to make api call to data -> dynamic with school if you want to scrape competition
    url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
    #makes get request and converts answer to json
    # url defines the page of all the information, request is made, and information is returned to data variable
    data = requests.get(url).json()
    #converts json to dataframe
    reviews =  pd.DataFrame(data['content']['reviews'])

    #aux function to apply regex and remove tags
    def remove_tags(x):
        return TAG_RE.sub('',x)
    reviews['review_body'] = reviews['body'].apply(remove_tags)
    reviews['school'] = school
    return reviews

In [7]:
# could you write this as a list comprehension? ;)
comments = []

for school in school_id_dict.keys():
    print(school)
    comments.append(get_comments_school(school))
comments = pd.concat(comments)

springboard
dataquest
syntax-technologies
ironhack
tripleten
colaberry
maven-analytics
udacity
brainstation
ccs-learning-academy
thinkful
general-assembly


In [8]:
comments.isna().sum()

id                    0
name                  0
anonymous             0
hostProgramName    4022
graduatingYear       57
isAlumni              1
jobTitle           2614
tagline               0
body                  0
rawBody               0
createdAt             0
queryDate             0
program             907
user                  0
overallScore          9
comments              0
overall               9
curriculum          176
jobSupport          791
review_body           0
school                0
dtype: int64

In [9]:
# transform years from float to integers
comments['graduatingYear'] = comments['graduatingYear'].astype('Int64')
comments.graduatingYear.value_counts()

2018    1466
2019    1106
2020    1009
2017     964
2021     768
2022     616
2016     421
2023     328
2015     183
2014      47
2012      13
2013      10
2011       1
Name: graduatingYear, dtype: Int64

In [10]:
comments.tagline.value_counts()

Great Experience                       30
Great experience                       23
Amazing experience                     23
Great experience!                      22
Amazing                                18
                                       ..
My best decision.                       1
A social and work oriented bootcamp     1
Very happy with my choice               1
Awsome                                  1
NULL                                    1
Name: tagline, Length: 6312, dtype: int64

In [11]:
# uniforming tag values in tagline
tagline_mapping = {
    'Great Experience': ['Great Experience', 'Great experience', 'Great experience!', 'Great Experience', 'Great'],
    'Amazing Experience': ['Amazing experience', 'Amazing']
}

comments['tagline'] = comments['tagline'].apply(lambda x: next((key for key, value in tagline_mapping.items() if x in value), x))

# check
print(comments['tagline'].value_counts())

Great Experience                         88
Amazing Experience                       54
Springboard Data Science Career Track    15
Excellent                                15
Life changing                            13
                                         ..
My best decision.                         1
A social and work oriented bootcamp       1
Very happy with my choice                 1
Awsome                                    1
NULL                                      1
Name: tagline, Length: 6307, dtype: int64


In [12]:
# columns body and rawBody contain uncleaned data and is replaceable by the cleaned review_body column.
# queryData contains the same info as createdAt.
comments.drop(columns={'body', 'rawBody', 'queryDate'}, inplace=True)

In [13]:
# column user is empty while column comments is a list of each comments for every review.
# We choose to not keep it as we already have thousands of single comments
comments.drop(columns={'user', 'comments'}, inplace=True)

In [14]:
comments['school_id'] = comments.school.map(school_id_dict)

In [15]:
#after having multiple issues when importing the comments.csv in SQL, we went 'a little' deeper with the cleaning...

In [16]:
comments.isna().sum()

id                    0
name                  0
anonymous             0
hostProgramName    4022
graduatingYear       57
isAlumni              1
jobTitle           2614
tagline               0
createdAt             0
program             907
overallScore          9
overall               9
curriculum          176
jobSupport          791
review_body           0
school                0
school_id             0
dtype: int64

In [17]:
comments.dropna(subset=['overallScore'], inplace=True)

In [18]:
comments.dropna(subset=['curriculum'], inplace=True)

In [19]:
comments.dropna(subset=['jobSupport'], inplace=True)

In [20]:
comments.drop(998, inplace=True)

In [21]:
comments['overallScore'] = pd.to_numeric(comments['overallScore'], errors='coerce')
comments['overallScore'].dropna(inplace=True)

In [22]:
comments['hostProgramName'].fillna('Not specified', inplace=True)

In [23]:
comments['jobTitle'].fillna('Not specified', inplace=True)

In [24]:
comments.dropna(subset='graduatingYear', inplace=True)

In [25]:
comments['curriculum'] = pd.to_numeric(comments['curriculum'], errors='coerce')

In [26]:
comments['jobSupport'] = pd.to_numeric(comments['jobSupport'], errors='coerce')

In [27]:
comments.columns

Index(['id', 'name', 'anonymous', 'hostProgramName', 'graduatingYear',
       'isAlumni', 'jobTitle', 'tagline', 'createdAt', 'program',
       'overallScore', 'overall', 'curriculum', 'jobSupport', 'review_body',
       'school', 'school_id'],
      dtype='object')

In [28]:
comments.drop(columns={'anonymous', 'overall', 'program', 'createdAt'}, inplace=True)

In [29]:
comments.columns

Index(['id', 'name', 'hostProgramName', 'graduatingYear', 'isAlumni',
       'jobTitle', 'tagline', 'overallScore', 'curriculum', 'jobSupport',
       'review_body', 'school', 'school_id'],
      dtype='object')

In [30]:
comments

,id,name,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,overallScore,curriculum,jobSupport,review_body,school,school_id
0,306638,Constance Sun,Not specified,2023,False,,Very accommodating. Take Springboard if you wa...,3.7,4.0,3.0,When I was looking at which bootcamp to do for...,springboard,11035
1,306549,Daniel Dluzynski,Not specified,2023,False,,Extensive and well built curriculum,4.3,5.0,4.0,This course is great for beginners. The curric...,springboard,11035
2,306505,Jonathan Chiu,Not specified,2023,False,,Join if you're looking to structure &amp; Netw...,4.0,4.0,4.0,"If you find yourself unsure of where to begin,...",springboard,11035
3,306504,Anonymous,Not specified,2023,False,,Join if you're looking to structure &amp; Netw...,4.0,4.0,4.0,"If you find yourself unsure of where to begin,...",springboard,11035
4,306451,Anonymous,Not specified,2023,True,UX/UI Design,Wonderful,4.3,5.0,4.0,Pros: I found the Springboard bootcamp to be i...,springboard,11035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,232579,Anonymous,UX/UI Design,2015,True,Not specified,GA is great on most levels,4.0,4.0,4.0,"Pros: Great environment, phenomenal network, a...",general-assembly,10761
982,232573,Carlye Cunniff,Not specified,2015,True,Not specified,"General Assembly UXDI an amazing experience, i...",4.3,4.0,4.0,I was beyond impressed by my experience at Gen...,general-assembly,10761
986,232490,Anonymous,Not specified,2015,True,,Incredible Learning Opportunity for Digital Ma...,5.0,5.0,5.0,I recently completed an Ad Optimization part t...,general-assembly,10761
991,232206,Anonymous,Not specified,2014,True,,Not The Best Bootcamp I Could Have Chosen,1.7,2.0,1.0,I wouldn't say that Bitmaker was a bad experie...,general-assembly,10761


In [31]:
comments.dtypes

id                   int64
name                object
hostProgramName     object
graduatingYear       Int64
isAlumni            object
jobTitle            object
tagline             object
overallScore       float64
curriculum         float64
jobSupport         float64
review_body         object
school              object
school_id            int64
dtype: object

In [32]:
from pandas import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])

    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school


    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in school_id_dict.items():
    print(school)
    a,b,c,d = get_school_info(school,id)

    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)

springboard
dataquest


syntax-technologies
ironhack
tripleten
colaberry
maven-analytics
udacity
brainstation
ccs-learning-academy
thinkful
general-assembly


In [33]:
locations_list

[      id description  state.id state.name state.abbrev state.keyword  \
 0  16013      Online         1     Online       Online        online   
 
         school  school_id  
 0  springboard      11035  ,
       id description  state.id state.name state.abbrev state.keyword  \
 0  16378      Online         1     Online       Online        online   
 
       school  school_id  
 0  dataquest      10683  ,
       id description  state.id state.name state.abbrev state.keyword  \
 0  18261      Online         1     Online       Online        online   
 
                 school  school_id  
 0  syntax-technologies      11797  ,
       id               description  country.id   country.name country.abbrev  \
 0  15901           Berlin, Germany        57.0        Germany             DE   
 1  16022       Mexico City, Mexico        29.0         Mexico             MX   
 2  16086    Amsterdam, Netherlands        59.0    Netherlands             NL   
 3  16088         Sao Paulo, Brazil        

In [34]:
locations = pd.concat(locations_list)
locations.isna().sum()

id                 0
description        0
state.id          10
state.name        10
state.abbrev      10
state.keyword     10
school             0
school_id          0
country.id        12
country.name      12
country.abbrev    12
city.id           12
city.name         12
city.keyword      12
dtype: int64

In [35]:
# eliminate redundant data
locations.drop(columns={
    'state.abbrev',
    'state.keyword',
    'country.abbrev',
    'city.keyword',
    'state.id',
    'city.id',
    'country.id'
}, inplace=True)

In [36]:
locations['country.name'] = locations['country.name'].apply(lambda value: 'Online' if pd.isna(value) else value)

In [37]:
locations['city.name'] = locations['city.name'].apply(lambda value: 'Online' if pd.isna(value) else value)

In [38]:
locations['state.name'].fillna(locations['country.name'], inplace=True)

In [39]:
locations.isna().sum()

id              0
description     0
state.name      0
school          0
school_id       0
country.name    0
city.name       0
dtype: int64

In [40]:
locations.columns

Index(['id', 'description', 'state.name', 'school', 'school_id',
       'country.name', 'city.name'],
      dtype='object')

In [41]:
courses = pd.concat(courses_list)

In [42]:
courses

,courses,school,school_id
0,Cyber Security Career Track,springboard,11035
1,Data Analytics Career Track,springboard,11035
2,Data Science Career Track,springboard,11035
3,Data Science Career Track Prep,springboard,11035
4,Front-End Web Development,springboard,11035
...,...,...,...
9,React Development (Short Course),general-assembly,10761
10,Software Engineering Bootcamp (Full-Time),general-assembly,10761
11,User Experience Design (Short Course),general-assembly,10761
12,User Experience Design Bootcamp (Full-Time),general-assembly,10761


In [43]:
badges = pd.concat(badges_list)
badges

,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,springboard,11035
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,springboard,11035
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,springboard,11035
0,Available Online,available_online,<p>School offers fully online courses</p>,dataquest,10683
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,dataquest,10683
0,Available Online,available_online,<p>School offers fully online courses</p>,syntax-technologies,11797
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,syntax-technologies,11797
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828


In [44]:
badges.drop(columns={'keyword'}, inplace=True)

In [45]:
badges.columns

Index(['name', 'description', 'school', 'school_id'], dtype='object')

In [46]:
text = '<p>School offers fully online courses</p>'
text.split('>')[1].split('<')[0]

'School offers fully online courses'

In [47]:
badges['description'] = badges['description'].apply(lambda text: text.split('>')[1].split('<')[0])

In [48]:
badges[badges['school'] == 'springboard']

,name,description,school,school_id
0,Available Online,School offers fully online courses,springboard,11035
1,Flexible Classes,School offers part-time and evening classes,springboard,11035
2,Job Guarantee,School guarantees job placement,springboard,11035


In [49]:
# any data cleaning still missing here? take a look at the description
schools = pd.concat(schools_list)
schools

,website,description,LogoUrl,school,school_id
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035
0,www.dataquest.io,"<span class=""truncatable""><p>Master data skill...",https://d92mrp7hetgfk.cloudfront.net/images/si...,dataquest,10683
0,www.syntaxtechs.com/,"<span class=""truncatable""><p>Syntax Technologi...",https://d92mrp7hetgfk.cloudfront.net/images/si...,syntax-technologies,11797
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,tripleten.com/?utm_source=referral&utm_medium=...,"<span class=""truncatable""><p>Tripleten changed...",https://d92mrp7hetgfk.cloudfront.net/images/si...,tripleten,11225
0,www.colaberry.com/,"<span class=""truncatable""><p>Colaberry offers ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,colaberry,11718
0,www.mavenanalytics.io/,"<span class=""truncatable""><p>Maven Analytics i...",https://d92mrp7hetgfk.cloudfront.net/images/si...,maven-analytics,11740
0,www.udacity.com/?utm_source=switchup&utm_mediu...,"<span class=""truncatable""><p>Udacity is the tr...",https://d92mrp7hetgfk.cloudfront.net/images/si...,udacity,11118
0,brainstation.io,"<span class=""truncatable""><p>BrainStation is t...",https://d92mrp7hetgfk.cloudfront.net/images/si...,brainstation,10571
0,ccslearningacademy.com/,"<span class=""truncatable""><p>TECH TRAINING BY ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ccs-learning-academy,11736


In [50]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(x):
  return TAG_RE.sub('',x)

In [51]:
schools['description'] = schools['description'].apply(remove_tags)
schools

,website,description,LogoUrl,school,school_id
0,www.springboard.com/?utm_source=switchup&utm_m...,Springboard is an online learning platform tha...,https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035
0,www.dataquest.io,Master data skills. Build your career.Dataques...,https://d92mrp7hetgfk.cloudfront.net/images/si...,dataquest,10683
0,www.syntaxtechs.com/,Syntax Technologies is the world’s leading onl...,https://d92mrp7hetgfk.cloudfront.net/images/si...,syntax-technologies,11797
0,www.ironhack.com/en,Ironhack is a global tech school with 9 campus...,https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,tripleten.com/?utm_source=referral&utm_medium=...,Tripleten changed their brand name from Practi...,https://d92mrp7hetgfk.cloudfront.net/images/si...,tripleten,11225
0,www.colaberry.com/,Colaberry offers instructor-led remote and sel...,https://d92mrp7hetgfk.cloudfront.net/images/si...,colaberry,11718
0,www.mavenanalytics.io/,"Maven Analytics is an online, guided learning ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,maven-analytics,11740
0,www.udacity.com/?utm_source=switchup&utm_mediu...,Udacity is the trusted market leader in talent...,https://d92mrp7hetgfk.cloudfront.net/images/si...,udacity,11118
0,brainstation.io,BrainStation is the global leader in digital s...,https://d92mrp7hetgfk.cloudfront.net/images/si...,brainstation,10571
0,ccslearningacademy.com/,TECH TRAINING BY TECH PROFESSIONALSCCS Learnin...,https://d92mrp7hetgfk.cloudfront.net/images/si...,ccs-learning-academy,11736


In [52]:
schools.columns

Index(['website', 'description', 'LogoUrl', 'school', 'school_id'], dtype='object')

In [53]:
# Create an empty list to store the data
course_prices_list = []

for school_name in school_list:
    url = f"https://www.switchup.org/bootcamps/{school_name.lower().replace(' ', '-')}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

    try:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all course__inner elements
            course_inners = soup.find_all('div', class_='course__inner')

            if course_inners:
                # Iterate over each course__inner element
                for course_inner in course_inners:

                    # Extract school name
                    school_name = school_name

                    # Extract course name
                    course_name = course_inner.find('h4').text.strip()

                    # Extract cost information
                    cost_div = course_inner.find('div', class_='cost')
                    if cost_div:
                        cost = cost_div.find('span').text.strip()
                    else:
                        cost = "Not found"

                    # Append data to the list
                    course_prices_list.append({
                        'school': school_name,
                        'course': course_name,
                        'price': cost})

            else:
                print(f"{school_name}: Not found")

    except requests.RequestException as e:
        print(f"Error for {school_name}: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Create a DataFrame from the list of dictionaries
course_prices = pd.DataFrame(course_prices_list)


In [54]:
course_prices.dtypes

school    object
course    object
price     object
dtype: object

In [55]:
course_prices.price.value_counts()

$4,500               5
$3,250               5
Not found            4
$3,950               4
$9,900               4
$1,077               4
$9,500               3
$16,450              3
$1,436               3
€7,500               3
$16,500              2
$2,154               2
$4,900               2
$9,700               2
$13,000              2
$12,000              2
$29                  2
$349                 2
$7,995               1
$16,000              1
$12,250              1
$8,000               1
$8,995               1
$6,500               1
$12,150              1
$11,995              1
$718                 1
$12,995              1
$7,499               1
$1,500   - $4,000    1
$1,999               1
$6,900               1
$7,900               1
€8,000               1
$5,900               1
$490                 1
$8,500               1
$3,500               1
Name: price, dtype: int64

In [56]:
# formatting the prices to make it easier to transform them in numbers
course_prices['price'] = course_prices['price'].replace('[\$,]', '', regex=True)

In [57]:
course_prices['price'] = course_prices['price'].replace('[\€,]', '', regex=True)

In [58]:
course_prices.price

0     Not found
1          9900
2          8500
3          9900
4           490
        ...    
69         4500
70         4500
71         4500
72         3500
73        16450
Name: price, Length: 74, dtype: object

In [59]:
course_prices

,school,course,price
0,Springboard,Front-End Web Development,Not found
1,Springboard,Cyber Security Career Track,9900
2,Springboard,Data Analytics Career Track,8500
3,Springboard,Data Science Career Track,9900
4,Springboard,Data Science Career Track Prep,490
...,...,...,...
69,General Assembly,User Experience Design (Short Course),4500
70,General Assembly,Front-End Web Development (Short Course),4500
71,General Assembly,Product Management (Short Course),4500
72,General Assembly,Visual Design (Short Course),3500


In [60]:
course_prices['school'] = course_prices['school'].str.lower().str.replace(' ','-')

In [61]:
school_id_dict

{'springboard': 11035,
 'dataquest': 10683,
 'syntax-technologies': 11797,
 'ironhack': 10828,
 'tripleten': 11225,
 'colaberry': 11718,
 'maven-analytics': 11740,
 'udacity': 11118,
 'brainstation': 10571,
 'ccs-learning-academy': 11736,
 'thinkful': 11098,
 'general-assembly': 10761}

In [62]:
course_prices['school_id'] = course_prices['school'].map(school_id_dict)
course_prices

,school,course,price,school_id
0,springboard,Front-End Web Development,Not found,11035
1,springboard,Cyber Security Career Track,9900,11035
2,springboard,Data Analytics Career Track,8500,11035
3,springboard,Data Science Career Track,9900,11035
4,springboard,Data Science Career Track Prep,490,11035
...,...,...,...,...
69,general-assembly,User Experience Design (Short Course),4500,10761
70,general-assembly,Front-End Web Development (Short Course),4500,10761
71,general-assembly,Product Management (Short Course),4500,10761
72,general-assembly,Visual Design (Short Course),3500,10761


In [63]:
comments.loc[comments['jobTitle'] == '', 'jobTitle'] = 'Not specified'

In [64]:
comments.dropna(subset='jobTitle', inplace=True)

In [65]:
comments.isnull().sum()

id                 0
name               0
hostProgramName    0
graduatingYear     0
isAlumni           0
jobTitle           0
tagline            0
overallScore       0
curriculum         0
jobSupport         0
review_body        0
school             0
school_id          0
dtype: int64

In [66]:
comments.drop(columns='name', inplace=True)

In [67]:
comments.drop(columns='school', inplace=True)

In [ ]:
#export of filtered comments with numerical columns to perform some queries

In [68]:
comments_numeric = comments.drop(columns={'hostProgramName', 'isAlumni', 'jobTitle', 'tagline', 'review_body'})

In [69]:
comments_numeric.to_csv('comments_numeric.csv', index=False)

In [70]:
comments.to_csv('comments.csv', index=False)
schools.to_csv('schools.csv', index=False)
badges.to_csv('badges', index=False)
locations.to_csv('locations.csv', index=False)
courses.to_csv('courses.csv', index=False)
course_prices.to_csv('course_prices.csv', index=False)

In [ ]:
#attempts to eliminate special charchters from object type columns: WIP

In [71]:
# trying to troubleshoot the huge loss of data when importing comments
trial_df = pd.read_csv('comments.csv')
print(trial_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6173 entries, 0 to 6172
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               6173 non-null   int64  
 1   hostProgramName  6173 non-null   object 
 2   graduatingYear   6173 non-null   int64  
 3   isAlumni         6173 non-null   bool   
 4   jobTitle         6167 non-null   object 
 5   tagline          6173 non-null   object 
 6   overallScore     6173 non-null   float64
 7   curriculum       6173 non-null   float64
 8   jobSupport       6173 non-null   float64
 9   review_body      6173 non-null   object 
 10  school_id        6173 non-null   int64  
dtypes: bool(1), float64(3), int64(3), object(4)
memory usage: 488.4+ KB
None


In [72]:
# Check for any missing values in the DataFrame
print("Missing values:\n", trial_df.isnull().sum())

Missing values:
 id                 0
hostProgramName    0
graduatingYear     0
isAlumni           0
jobTitle           6
tagline            0
overallScore       0
curriculum         0
jobSupport         0
review_body        0
school_id          0
dtype: int64


In [73]:
# Check for unique values in id column
trial_df.id.value_counts(ascending=False)

306638    1
257068    1
256971    1
256973    1
256974    1
         ..
267640    1
267742    1
267864    1
267895    1
231691    1
Name: id, Length: 6173, dtype: int64

In [81]:
# Check for the presence of special characters or unexpected data
for column in trial_df.columns:
    # Skip numeric columns
    if pd.api.types.is_numeric_dtype(trial_df[column]):
        continue

    special_characters = trial_df[trial_df[column].astype(str).str.contains(r'[^a-zA-Z0-9\s]', na=False)]
    if not special_characters.empty:
        print(f"Special characters in {column}:\n{special_characters}")

Special characters in hostProgramName:
          id hostProgramName  graduatingYear  isAlumni  \
918   267361    UX/UI Design            2020     False   
919   267344    UX/UI Design            2020     False   
924   267000    UX/UI Design            2020     False   
925   266956    UX/UI Design            2020     False   
926   266952    UX/UI Design            2020     False   
...      ...             ...             ...       ...   
6131  236669    UX/UI Design            2016      True   
6132  244743    UX/UI Design            2015      True   
6135  244731    UX/UI Design            2016      True   
6153  244545    UX/UI Design            2016      True   
6168  232579    UX/UI Design            2015      True   

                    jobTitle  \
918            Not specified   
919   UX designer/consultant   
924     UX Research/Designer   
925          UX/ UI Designer   
926            Not specified   
...                      ...   
6131     Creative Specialist   
6132    

In [ ]:
#it seems that the problem in importin is just that there are too many special charachters and the rows containing them are automatically dropped.

In [80]:
special_characters = r'[!@#$%^&*()_+{}|:"<>?\\[\];,./\']'

# Identify string columns
string_columns = trial_df.select_dtypes(include='object').columns

for columns in string_columns:
    trial_df[column] = trial_df[column].astype(str).apply(lambda x: x.replace(special_characters, ''))

# Display the updated DataFrame
print(trial_df)


          id hostProgramName  graduatingYear  isAlumni  \
0     306638   Not specified            2023     False   
1     306549   Not specified            2023     False   
2     306505   Not specified            2023     False   
3     306504   Not specified            2023     False   
4     306451   Not specified            2023      True   
...      ...             ...             ...       ...   
6168  232579    UX/UI Design            2015      True   
6169  232573   Not specified            2015      True   
6170  232490   Not specified            2015      True   
6171  232206   Not specified            2014      True   
6172  231691   Not specified            2013      True   

                                    jobTitle  \
0                              Not specified   
1                              Not specified   
2                              Not specified   
3                              Not specified   
4                               UX/UI Design   
...            

In [82]:
duplicates = trial_df[trial_df.duplicated()]
if not duplicates.empty:
    print("Duplicate rows:\n", duplicates)

In [84]:
# Identify string columns
string_columns = trial_df.select_dtypes(include='object').columns
string_columns

Index(['hostProgramName', 'jobTitle', 'tagline', 'review_body', 'school_id'], dtype='object')

In [ ]:
#df.to_csv('trial.csv', index=False)

In [ ]:
df.dtypes

id                   int64
hostProgramName     object
graduatingYear       int64
isAlumni              bool
jobTitle            object
tagline             object
overallScore       float64
curriculum         float64
jobSupport         float64
review_body         object
school_id            int64
dtype: object